# HW

In [1]:
import numpy as np
import pandas as pd
import operator

##### Загрузите датасет titanic.csv

In [2]:
df_titanic = pd.read_csv('titanic.csv')

In [3]:
df_titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df_titanic.shape

(891, 12)

In [5]:
df_titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


> Обратите внимание на разное количество значений переменных
Если это так - то у вас пропуски в данных. Такое бывает. 
Например в этом датасете не получилось выяснить всех подробностей у некоторых людей в виду утери данных, невозможности узнать или восстановить их

In [6]:
df_titanic.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


> Давайте возьмем для начала только несколько признаков, позже - вы увеличите их количество и сделаете соответствующую обработку.

In [7]:
columns = ['Sex', 'Age', 'Pclass', 'Survived']

In [8]:
simple_train = df_titanic[columns].loc[:500]
simple_test = df_titanic[columns].loc[500:]

> мы пока не умеем на уровне алгоритма работать с пропущенными значениями. 
Представьте вам дают 5-мерный вектор у которого заполнены только три компоненты и другой такой же вектор но у него заполнены лишь две  компоненты.  Как померить их сходство????

distance_between( [1,0,1, NaN, NaN], [NaN, 1, 1, NaN, NaN] )


> один из вариантов - заполнить какими то значениями - например если это возраст - то заполнить средним по датасету или нулем.

>> на будущее - в sklearn есть такой модуль - http://scikit-learn.org/stable/modules/impute.html но сейчас можете заполнить пропуски средствами pandas - метод fillna()

##### Заполните пропущенные значения чем нибудь

In [9]:
#simple_train[np.isnan(simple_train['Age'])]

In [10]:
simple_train = simple_train.fillna(simple_train.Age.mean())
simple_test = simple_test.fillna(simple_train.Age.mean())
# можно inplace делать такое, а можно переприсваивать.

> мы чемпионы. Но мы понимаем, что алгоритм у нас метрический - он не понимает как померить близость векторов в которых содержатся строки

distance( [1,1,0,'мужик'], [1, 0, 1, 'принцесска'] )

> поэтому давайте заведем признак is_man который будет принимать значение 0 или 1.

##### Заведите бинарный признак отвечающий за пол. Строковый удалите

хинт - есть метод apply()

In [11]:
d = {'male' : 1, 'female' : 0}
simple_train.Sex = simple_train.Sex.map(d)
simple_test.Sex = simple_test.Sex.map(d)

##### Сформируйте трейн и тест выборку, где в X лежат признаки 'Sex', 'Age', 'Pclass', а в у - Survived

Результат - numpy array

hint - у pandas есть метод drop() и атрибут values; Пощупайте их

In [12]:
X_train = simple_train.drop('Survived',axis=1).values
X_test = simple_test.drop('Survived',axis=1).values
y_train = simple_train.drop(['Sex', 'Age', 'Pclass'],axis=1).values
y_test = simple_test.drop(['Sex', 'Age', 'Pclass'],axis=1).values

In [13]:
assert isinstance(X_train, np.ndarray)
assert isinstance(X_test, np.ndarray)
assert isinstance(y_train, np.ndarray)
assert isinstance(y_test, np.ndarray)

In [14]:
assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]

In [15]:
assert not np.any(np.isnan(X_train)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'
assert not np.any(np.isnan(X_test)), 'у вас в данных есть пропущенные значения. Заполните их или удалите'

##### Реализуйте KNN. Сами, как делали это на лекции. 

Можете улучшить его.
(например использовать kd-tree для быстрого поиска ближайших соседей, 
 или обходить соседей и класть расстояния в мин-кучу - тогда вы быстрее будете набирать их, без сортировки.)

In [204]:
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
x.items()

dict_items([(1, 2), (3, 4), (4, 3), (2, 1), (0, 0)])

In [215]:
import operator
x = {1: 2, 3: 4, 4: 3, 2: 1, 0: 0}
sorted_x = sorted(x.items(), key=operator.itemgetter(1))
sorted_x


[(0, 0), (2, 1), (1, 2), (4, 3), (3, 4)]

In [31]:
i=0
for (sex,age, pclass) in X_train:
    i=i+1
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [80]:
class KNN:
    """
    simple KNN classifier with O(n) complexity for memory,
    and O(n log n) complexity - for nn search.
    """    
    def __init__(self, n):
        self.n = n
        
    def fit(self, X, y):
        self.x = X
        self.y = y
    
    def predict(self, X):
        y_hat = []
        i = 0
        assert len(X.shape) == 2
        for (sex,age, pclass) in X: 
            dist = {}
            idx = []
            count = np.array([])
            for i in range(X.shape[0]):
                dist[i] = np.sqrt( (sex-self.x[i][0]) ** 2 + (age-self.x[i][1]) ** 2 + (pclass - self.x[i][2]) ** 2)
            sorted_dist = sorted(dist.items(), key=operator.itemgetter(1))
            for i in range(self.n):
                idx.append(sorted_dist[i][0]) #idx - массив из индексов n элементов
            for i in range(len(idx)):
                count = np.append(count,self.y[idx[i]][0])
            count_1 = 0
            count_0 = 0    
            predict = 0
            for i in range(len(count)):
                if count[i] == 0:
                    count_0 +=1
                else:
                    count_1 +=1
            if count_0 <= count_1:
                predict = 1
            else:
                predict = 0
            y_hat.append(predict)
        return np.array(y_hat)

In [81]:
knn = KNN(5)

knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)

In [82]:
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,

##### посчитайте руками (не используя sklearn) качество вашего предсказания

In [83]:
acc = np.mean(y_pred == y_test)
print('доля правильных ответов:', acc)

доля правильных ответов: 0.5587025202608564


In [84]:
np.unique(y_pred)

array([0, 1])

> Мы получили бейслайн решение. Оно должно давать скорр в районе 0.76

____

In [78]:
### можете проверить себя

from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.7544757033248082


In [79]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

clf=KNeighborsClassifier(n_neighbors=5)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

np.unique(y_pred)

print(accuracy_score(y_test, y_pred))

0.7723785166240409


/home/twsea/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  """


##### TODO продолжение домашки